In [ ]:
!pip install geopandas
!pip install contextily
import pandas as pd
import geopandas
import contextily as ctx

In [ ]:
import os
!pip install rtree
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
os.chdir("/content/spatialindex-src-1.8.5")
!./configure
!make
!make install
!ldconfig

In [ ]:
stradario = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/stradario.csv")
bike_areesosta = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/bike_areesosta.zip")
quartieri = geopandas.read_file("/content/drive/MyDrive/Colab Notebooks/geopandas-tutorial-master/ds964_nil_wm.zip")

In [ ]:
bike_areesosta

,id_amat,id_via,num_civico,municipio,stato,categoriev,numero_man,stalli_per,stalli_tot,tipomanufa,ubicazione,geometry
0,2,5339,0,5,Esistente,Velocipedi,3,5,15,Verona,Marciapiede,POINT (9.17821 45.42991)
1,4,1545,21,9,Esistente,Velocipedi,2,5,10,0_Non precisato,Alla quota della carreggiata,POINT (9.16829 45.51697)
2,8,6330,42,7,Esistente,Velocipedi,1,5,5,Redondo Inox,Cortile o altro spazio collegato al civico,POINT (9.08480 45.47628)
3,9,7165,4,8,Esistente,Velocipedi,1,5,5,Redondo Inox,Cortile o altro spazio collegato al civico,POINT (9.15673 45.48228)
4,10,6660,5,6,Esistente,Velocipedi,1,4,4,0_Non precisato,Marciapiede,POINT (9.12054 45.45506)
...,...,...,...,...,...,...,...,...,...,...,...,...
1517,3232,1242,72,9,Esistente,Velocipedi,3,4,12,Triangolare,Cortile o altro spazio collegato al civico,POINT (9.16586 45.50530)
1518,3233,1242,72,9,Esistente,Velocipedi,5,2,10,Archetto Milano _ Acciaio inox _ Passo cm 80,Cortile o altro spazio collegato al civico,POINT (9.16628 45.50584)
1519,3234,1242,72,9,Esistente,Velocipedi,6,2,12,Archetto Milano _ Acciaio inox _ Passo cm 80,Cortile o altro spazio collegato al civico,POINT (9.16634 45.50568)
1520,3235,1242,72,9,Esistente,Velocipedi,29,2,58,Archetto Milano _ Acciaio inox _ Passo cm 80,Cortile o altro spazio collegato al civico,POINT (9.16633 45.50532)


- una lista (elenco) delle aree di sosta, indicando il nome della via, il numero civico e il numero di posti totali per le biciclette


In [ ]:
bike_areesosta["id_via"] = bike_areesosta["id_via"].astype(str)
bike_areesosta.id_via.dtypes

In [ ]:
stradario["CODICE_VIA"] = stradario["CODICE_VIA"].astype(str)
stradario.CODICE_VIA.dtypes

In [ ]:
areesosta_vie = pd.merge(bike_areesosta, stradario, how = "inner", left_on = "id_via", right_on = "CODICE_VIA")
areesosta_vie[["AREA", "DESCRIZIONE_VIA", "CIVICO", "stalli_tot"]]

- una lista (elenco) analogo al precedente ma con anche  l'indicazione del quartiere


In [ ]:
areesosta_quartieri = geopandas.sjoin(areesosta_vie, quartieri, op = "within", how = "left")[["AREA", "DESCRIZIONE_VIA", "CIVICO", "stalli_tot", "NIL"]]
areesosta_quartieri

- i nomi dei quartieri in cui non ci sono aree di sosta;


In [ ]:
quartieri_senza = quartieri[~quartieri.crosses(bike_areesosta.unary_union)]
quartieri_senza

- una mappa di Milano con l'indicazione dei quartieri in cui non ci sono aree di sosta


In [ ]:
ax = quartieri_senza.to_crs(epsg=3857).plot(figsize = (12,20), facecolor = "none", edgecolor = "k")
ctx.add_basemap(ax)

- il nome del quartiere con più aree di sosta;


In [ ]:
areesosta_quartieri2 = geopandas.sjoin(areesosta_vie, quartieri, op = "within", how = "left")[["AREA", "DESCRIZIONE_VIA", "CIVICO", "stalli_tot", "NIL", "id_amat"]]
areesosta_quartieri2.drop_duplicates("id_amat", inplace=True)
areesosta_quartieri3 = areesosta_quartieri2.groupby("NIL")["DESCRIZIONE_VIA"].count()
areesosta_quartieri4 = areesosta_quartieri3[areesosta_quartieri3.values == areesosta_quartieri3.values.max()]
areesosta_quartieri4

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if self.run_code(code, result):


NIL
DUOMO    132
Name: DESCRIZIONE_VIA, dtype: int64

In [ ]:
areesosta_quartieri4.reset_index()

,NIL,DESCRIZIONE_VIA
0,DUOMO,132


In [ ]:
quartieri

,ID_NIL,NIL,Valido_dal,Valido_al,Fonte,Shape_Leng,Shape_Area,OBJECTID,geometry
0,48,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,8723.368714,2.406306e+06,89,"POLYGON ((9.15422 45.43775, 9.15419 45.43707, ..."
1,64,TRENNO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,3309.998800,4.896921e+05,90,"POLYGON ((9.10623 45.49016, 9.10295 45.48939, ..."
2,67,PORTELLO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,3800.750663,9.096022e+05,91,"POLYGON ((9.15636 45.48785, 9.15724 45.48721, ..."
3,81,BOVISASCA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,7105.469715,1.578028e+06,92,"POLYGON ((9.16803 45.52234, 9.16687 45.52027, ..."
4,84,PARCO NORD,05/02/2020,Vigente,Milano 2030 - PGT Approvato,11741.717005,1.532331e+06,93,"POLYGON ((9.20040 45.52848, 9.20055 45.52828, ..."
...,...,...,...,...,...,...,...,...,...
83,68,PAGANO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5221.836696,1.289686e+06,172,"POLYGON ((9.17174 45.47619, 9.17150 45.47595, ..."
84,2,BRERA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,5461.667196,1.637333e+06,173,"POLYGON ((9.19165 45.46906, 9.19105 45.46843, ..."
85,33,ROGOREDO - SANTA GIULIA,05/02/2020,Vigente,Milano 2030 - PGT Approvato,6012.496435,1.200116e+06,174,"POLYGON ((9.24044 45.44427, 9.24084 45.44424, ..."
86,17,ADRIANO,05/02/2020,Vigente,Milano 2030 - PGT Approvato,9276.875747,2.433358e+06,175,"POLYGON ((9.26729 45.51234, 9.26701 45.51241, ..."


- una mappa di Milano con l'indicazione del quartiere con più aree di sosta;


In [ ]:
quartiere_piu = quartieri[quartieri["NIL"] == "DUOMO"]
#ax = areesosta_quartieri4.to_crs(epsg=3857).plot(figsize =(12,20), facecolor = "none", edgecolor = "k")
#ctx.add_basemap(ax)
quartiere_piu

- i nomi dei quartieri con un numero di aree di sosta minori della media;
